# Request School Profile Data from GigaMaps APIs

![Giga](https://github.com/unicef/giga-maps-api/blob/main/giga-logo.jpg?raw=true)

## `SchoolProfile` Schema

A structure describing the profile of a school within the Giga data model.

### Fields

| Field                         | Type     | Required | Description                                      | Possible Values                          |
|------------------------------|----------|----------|--------------------------------------------------|-------------------------------------------|
| `giga_id_school`             | string   | Yes      | A unique identifier for the school               | UUID string (e.g. `"6395a931-d652-3bce-97af-72918cb4259b"`) |
| `school_area_type`           | int      | Yes      | Type of area where the school is located         | e.g. `1` for rural, `2` for urban, etc.    |
| `country_iso3_code`          | string   | Yes      | ISO 3166-1 alpha-3 country code                  | e.g. `"KEN"`, `"BRA"`, `"IND"`            |
| `admin1`                     | string   | Yes      | First-level administrative division              | e.g. `"Nairobi"`, `"Rio de Janeiro"`       |
| `admin2`                     | string   | Yes      | Second-level administrative division             | e.g. `"Westlands"`, `"Rio de Janeiro"`     |
| `connectivity`               | boolean  | Yes      | Indicates if the school has connectivity         | `true` or `false`                         |
| `connectivity_RT`           | boolean  | No       | Real-time connectivity status                    | `true`, `false`, or `null`                |
| `connectivity_RT_datasource`| string   | Yes      | Source of real-time connectivity data            | e.g. `"gigameter"`, `"ISP_FEED"`, or `null` |
| `connectivity_type`         | string   | No       | Type of connectivity technology used             | e.g. `"Fiber"`, `"ADSL"`, `"HDSL"`        |
| `school_data_source`        | string   | No       | Unique identifier for the data source            | e.g. `"MOE_2025"`, `"ISP_Survey"`, or `null` |

### Example

```json
{
  "giga_id_school": "6395a931-d652-3bce-97af-72918cb4259b",
  "school_area_type": "urban",
  "country_iso3_code": "BRA",
  "admin1": "Rio de Janeiro",
  "admin2": "Rio de Janeiro",
  "connectivity": true,
  "connectivity_RT": true,
  "connectivity_RT_datasource": null,
  "connectivity_type": "{ADSL,NULL}",
  "school_data_source": null
}

Note than in some cases attributes will be written in local language

In [1]:
# Install libraries as needed

import requests
import pandas as pd
import geopandas as gpd

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Assign the the ISO code for the country you would like school location data for as a string to country_code variable.
# The list of countries available on GigaMaps (with ISO codes) can be downloaded using the GigaMaps Country API. https://maps.giga.global/docs/explore-api

country_code = "BIH"

In [ ]:
# Assign your unique API key as a string to variable api_key (if you don't have an API key, you can request one from Giga Maps)
api_key = ""

In [ ]:
# URL for School Location API endpoint (using selected country)
BASE_URL = f"https://uni-ooi-giga-maps-service.azurewebsites.net/api/v1/schools_profile/?country_iso3_code={country_code}"

#Headers
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {api_key}"
}

In [ ]:
# Parameters (see Giga Maps API docs)
# Requests need to be paginated- split up into multiple requests to prevent overloading the API
page = 1

# Current max page size is 1000, meaning each request will return 100 entries 
size = 1000

In [ ]:
# Creates a list to store all the responses from API requests
all_schools = []

# Loop through pages until all school location data has been collected
while True:
    params = {
        "page": page,
        "size": size
    }
    
   # Request schools from the API endpoint 
    response = requests.get(BASE_URL, headers=headers, params=params)

    # Ensure that the response from the API is correct. With an incorrect response, no more requests will be sent
    if response.status_code != 200:
        print(f"Request failed at page {page}: {response.text}")
        break

    # Extract the schools returned from the API
    data = response.json()['data']

    # Check that the most recent request receives data in the response
    if not data or len(data) == 0:
        print(f"No more data returned at page {page}.")
        break

   #Continuously add newly fetched schools to the all_schools list
    all_schools.extend(data)
    clear_output(wait=True)
    display(f"Fetched page {page}, items received: {len(data)}")
    page += 1

In [ ]:
#Convert to pandas DataFrame
school_profile_data = gpd.GeoDataFrame(all_schools)

In [ ]:
# Save the filtered data to a GeoJSON file
school_profile_data.to_file("school_profile_data.geojson", driver="GeoJSON")